In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("salesdaily.csv")
df

,datum,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06,Year,Month,Hour,Weekday Name
0,01-02-2014,0.00,3.670,3.40,32.40,7.0,0.0,0.0,2.00,2014,1,248,Thursday
1,01-03-2014,8.00,4.000,4.40,50.60,16.0,0.0,20.0,4.00,2014,1,276,Friday
2,01-04-2014,2.00,1.000,6.50,61.85,10.0,0.0,9.0,1.00,2014,1,276,Saturday
3,01-05-2014,4.00,3.000,7.00,41.10,8.0,0.0,3.0,0.00,2014,1,276,Sunday
4,01-06-2014,5.00,1.000,4.50,21.70,16.0,2.0,6.0,2.00,2014,1,276,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,10-04-2019,7.34,5.683,2.25,22.45,13.0,0.0,1.0,1.00,2019,10,276,Friday
2102,10-05-2019,3.84,5.010,6.00,25.40,7.0,0.0,0.0,0.33,2019,10,276,Saturday
2103,10-06-2019,4.00,11.690,2.00,34.60,6.0,0.0,5.0,4.20,2019,10,276,Sunday
2104,10-07-2019,7.34,4.507,3.00,50.80,6.0,0.0,10.0,1.00,2019,10,276,Monday


In [3]:
df_new = df.melt(id_vars=['datum', 'Year', 'Month', 'Hour', 'Weekday Name'], var_name='Drug', value_name='Quantity')
df_new.head()

,datum,Year,Month,Hour,Weekday Name,Drug,Quantity
0,01-02-2014,2014,1,248,Thursday,M01AB,0.0
1,01-03-2014,2014,1,276,Friday,M01AB,8.0
2,01-04-2014,2014,1,276,Saturday,M01AB,2.0
3,01-05-2014,2014,1,276,Sunday,M01AB,4.0
4,01-06-2014,2014,1,276,Monday,M01AB,5.0


In [4]:
df_new.shape

(16848, 7)

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_new['Weekday Name'] = le.fit_transform(df_new['Weekday Name'])
df_new['Drug'] = le.fit_transform(df_new['Drug'])
df_new

,datum,Year,Month,Hour,Weekday Name,Drug,Quantity
0,01-02-2014,2014,1,248,4,0,0.00
1,01-03-2014,2014,1,276,0,0,8.00
2,01-04-2014,2014,1,276,2,0,2.00
3,01-05-2014,2014,1,276,3,0,4.00
4,01-06-2014,2014,1,276,1,0,5.00
...,...,...,...,...,...,...,...
16843,10-04-2019,2019,10,276,0,7,1.00
16844,10-05-2019,2019,10,276,2,7,0.33
16845,10-06-2019,2019,10,276,3,7,4.20
16846,10-07-2019,2019,10,276,1,7,1.00


In [6]:
train = df_new[df_new['Year'] < 2019]
test = df_new[df_new['Year'] >= 2019]

print(train.shape)
print(test.shape)

(14600, 7)
(2248, 7)


In [7]:
X_train = train.drop(['Hour','Quantity','datum'],axis = 1)
y_train = train['Quantity']

X_test = test.drop(['Hour','Quantity','datum'],axis = 1)
y_test = test['Quantity']

In [8]:
X_train

,Year,Month,Weekday Name,Drug
0,2014,1,4,0
1,2014,1,0,0
2,2014,1,2,0
3,2014,1,3,0
4,2014,1,1,0
...,...,...,...,...
16562,2018,12,4,7
16563,2018,12,0,7
16564,2018,12,2,7
16565,2018,12,3,7


In [9]:
from sklearn.ensemble import RandomForestRegressor
reg_rf = RandomForestRegressor(max_depth=2, random_state=0)
reg_rf.fit(X_train, y_train)


RandomForestRegressor(max_depth=2, random_state=0)

In [10]:
from sklearn.metrics import mean_squared_error
rf_pred = reg_rf.predict(X_test)
mse = mean_squared_error(y_test,rf_pred)
rmse = np.sqrt(mse)
rmse

np.float64(8.546336651758923)

In [11]:
import xgboost as xgb
reg = xgb.XGBRegressor(n_estimators = 1000,early_stopping_rounds = 50, learning_rate = 0.005)
reg.fit(X_train,y_train,
       eval_set = [(X_train,y_train),(X_test,y_test)],
       verbose = 10)
y_pred =reg.predict(X_test)
rmse = np.sqrt(mse)
print(rmse)

[0]	validation_0-rmse:10.89225	validation_1-rmse:10.62390
[10]	validation_0-rmse:10.51182	validation_1-rmse:10.28430
[20]	validation_0-rmse:10.15410	validation_1-rmse:9.97270
[30]	validation_0-rmse:9.81910	validation_1-rmse:9.68125
[40]	validation_0-rmse:9.50571	validation_1-rmse:9.40890
[50]	validation_0-rmse:9.21286	validation_1-rmse:9.15473
[60]	validation_0-rmse:8.93952	validation_1-rmse:8.91753
[70]	validation_0-rmse:8.68464	validation_1-rmse:8.69605
[80]	validation_0-rmse:8.44708	validation_1-rmse:8.48342
[90]	validation_0-rmse:8.22562	validation_1-rmse:8.28514
[100]	validation_0-rmse:8.01821	validation_1-rmse:8.10528
[110]	validation_0-rmse:7.82532	validation_1-rmse:7.93878
[120]	validation_0-rmse:7.64642	validation_1-rmse:7.78442
[130]	validation_0-rmse:7.48036	validation_1-rmse:7.64178
[140]	validation_0-rmse:7.32654	validation_1-rmse:7.51044
[150]	validation_0-rmse:7.18403	validation_1-rmse:7.39044
[160]	validation_0-rmse:7.05218	validation_1-rmse:7.27646
[170]	validation_0-r

In [12]:
param_grid = {
    'n_estimators': [100, 500, 1000],    # Number of trees in the forest
    'learning_rate': [0.01, 0.1, 0.2],   # Learning rate
    'max_depth': [3, 5, 7],               # Maximum depth of each tree
    'subsample': [0.8, 1.0],              # Subsample ratio of the training instances
    'colsample_bytree': [0.8, 1.0]        # Subsample ratio of columns when constructing each tree
}

In [13]:
import xgboost as xgb

In [14]:
xgb = xgb.XGBRegressor(random_state=42)
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(xgb, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             param_grid={'colsample_bytree': [0.8, 1.0],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 500, 1000],
                         'subsample': [0.8, 1.0]},
             scoring='neg_mean_squared_error')

In [15]:
print("Best Hyperparameters:", grid_search.best_params_)
best_xgb = grid_search.best_estimator_
y_pred = best_xgb.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("Test Root Mean Squared Error:", rmse)

Best Hyperparameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Test Root Mean Squared Error: 6.089615455974572


In [16]:
test['Quantity Predictions'] = y_pred
test

C:\Users\telan\AppData\Local\Temp\ipykernel_17328\3726821739.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Quantity Predictions'] = y_pred


,datum,Year,Month,Hour,Weekday Name,Drug,Quantity,Quantity Predictions
1825,01-01-2019,2019,1,276,5,0,0.00,4.578609
1826,01-02-2019,2019,1,276,6,0,5.33,4.905593
1827,01-03-2019,2019,1,276,4,0,4.33,4.616585
1828,01-04-2019,2019,1,276,0,0,7.00,5.424590
1829,01-05-2019,2019,1,276,2,0,8.01,5.555935
...,...,...,...,...,...,...,...,...
16843,10-04-2019,2019,10,276,0,7,1.00,3.118207
16844,10-05-2019,2019,10,276,2,7,0.33,3.901338
16845,10-06-2019,2019,10,276,3,7,4.20,4.444452
16846,10-07-2019,2019,10,276,1,7,1.00,3.483537


In [17]:
def predict_sales(start_date,end_date,drug):
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    df_test = pd.DataFrame(index=dates)
    df_test['Year'] = df_test.index.year
    df_test['Month'] = df_test.index.month
    df_test['Weekday Name'] = df_test.index.weekday
    df_test['Drug'] = drug
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    df_test['Weekday Name'] = le.fit_transform(df_test['Weekday Name'])
    df_test['predicted_quantity'] = reg.predict(df_test)
    return df_test

In [18]:
predict_sales(start_date = '2024-12-01',end_date = '2024-12-31',drug = 4)

,Year,Month,Weekday Name,Drug,predicted_quantity
2024-12-01,2024,12,6,4,9.471975
2024-12-02,2024,12,0,4,9.215612
2024-12-03,2024,12,1,4,9.075402
2024-12-04,2024,12,2,4,8.035041
2024-12-05,2024,12,3,4,6.704865
2024-12-06,2024,12,4,4,8.383363
2024-12-07,2024,12,5,4,8.994246
2024-12-08,2024,12,6,4,9.471975
2024-12-09,2024,12,0,4,9.215612
2024-12-10,2024,12,1,4,9.075402


In [19]:
predict_sales(start_date = '2024-10-01',end_date = '2024-10-31',drug = 0)

,Year,Month,Weekday Name,Drug,predicted_quantity
2024-10-01,2024,10,1,0,4.855311
2024-10-02,2024,10,2,0,5.423892
2024-10-03,2024,10,3,0,5.631983
2024-10-04,2024,10,4,0,4.605959
2024-10-05,2024,10,5,0,4.701528
2024-10-06,2024,10,6,0,4.609930
2024-10-07,2024,10,0,0,4.566560
2024-10-08,2024,10,1,0,4.855311
2024-10-09,2024,10,2,0,5.423892
2024-10-10,2024,10,3,0,5.631983
